### 필요모듈 업로드

In [2]:
!pip install konlpy
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import scipy as sp
import numpy as np
from scipy.sparse import csr_matrix
from scipy.spatial.distance import cosine

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
  Using cached JPype1-1.4.1-cp311-cp311-win_amd64.whl (345 kB)


### 한글 형태소 분석기 로드(토크나이저)

In [3]:
t=Okt()

### 데이터 추출 및 리스트화(한글질문데이터)

In [7]:
df=pd.read_csv('./qadata0.csv',encoding='cp949',header=None)  ###위치를 어떻게 해야되나
df=df[0]
df=df[:450]  
contents=df.values.tolist()

## user질문입력 =>실습부분

In [8]:
target_q_input= '무선 키 보드 키 스킨 교체 또는 신규 신청 애데 헌 조치 내용 은 어떻게도 ㅣ 나요 ?'#예시 질문을 지우고, 여기에  질문을 입력해주세요
contents.append(target_q_input)

### 한글 형태소 단위로 tokenize

In [9]:
contents_tokens=[t.morphs(row) for row in contents]
print('토큰나이징\n', contents_tokens[-1])

토큰나이징
 ['무선', '키', '보드', '키', '스킨', '교체', '또는', '신규', '신청', '애데', '헌', '조치', '내용', '은', '어떻게도', 'ㅣ', '나요', '?']


### 형태소로 나눈 토큰을 띄어쓰기로 구분후 한문장으로 붙이기 + 예시결과출력

In [10]:
def token_to_fullsent(contents_tokens):
    ttok_sentence_forVectorize=[]

    for content in contents_tokens:
        sentence=''
        for word in content:
            sentence=sentence+' '+word
        

        ttok_sentence_forVectorize.append(sentence)
    return ttok_sentence_forVectorize


tok_sentence_forVectorize=token_to_fullsent(contents_tokens)
print(tok_sentence_forVectorize[-1])

 무선 키 보드 키 스킨 교체 또는 신규 신청 애데 헌 조치 내용 은 어떻게도 ㅣ 나요 ?


### tf-idf vectorize 

### 문장들의 리스트를 임베딩하기위해 벡터화합니다 

 - TF-IDF 방식으로 단어의 가중치를 조정한 BOW 인코딩 벡터를 만든다.
 
 - BOW 인코딩¶

문서를 숫자 벡터로 변환하는 가장 기본적인 방법은 BOW(Bag of Words) 인코딩 방법이다. BOW 인코딩 방법에서는 전체 문서 {d1,d2,…,dn}
 를 구성하는 고정된 단어장(vocabulary) {t1,t2,…,tm}
 를 만들고 di
라는 개별 문서에 단어장에 해당하는 단어들이 포함되어 있는지를 표시하는 방법이다.

xi,j=문서 di내의 단어 tj의 출현 빈도
또는

xi,j={0,1.만약 단어 tj가 문서 di 안에 없으면만약 단어 tj가 문서 di 안에 있으면

In [11]:
vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')
vec=vectorizer.fit_transform(tok_sentence_forVectorize)
num_samples, num_features =vec.shape
print(num_samples, num_features)


451 174


### 타겟질문 임베딩

In [12]:
target_q=[tok_sentence_forVectorize[-1]] #임베딩하기위해 리스트 변환(임의 DB에서 가져오는 코드)

target_q_vec = vectorizer.transform(target_q)

#테스트 케이스
print("타겟 질문의 임베딩 결과: \n", target_q_vec)
print(type(target_q_vec))

타겟 질문의 임베딩 결과: 
   (0, 139)	0.12361526252289011
  (0, 99)	0.5123261711956304
  (0, 93)	0.5123261711956304
  (0, 87)	0.10911789669457254
  (0, 85)	0.10760275747635727
  (0, 83)	0.24933508359361803
  (0, 68)	0.1641730012121168
  (0, 59)	0.21600193051953384
  (0, 48)	0.10513975531543152
  (0, 30)	0.12866326538563497
  (0, 27)	0.5123261711956304
  (0, 17)	0.1020474633032692
<class 'scipy.sparse._csr.csr_matrix'>


### 벡터화 된 matrix벡터에 대해서 score의 기반이 되는 내적거리 함수 정의(유클리디언 거리)

In [13]:
def dist_raw_eculidian(v1,v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())


### 코사인 유사도로 score계산하는 경우

In [14]:
def dist_cosine_similarity(v1, v2):
    # Ensure that both input matrices are in Compressed Sparse Row (CSR) format
    if not isinstance(v1, csr_matrix) or not isinstance(v2, csr_matrix):
        raise ValueError("Input matrices must be in CSR format")
    
    dense_matrix1 = v1.toarray().ravel() if hasattr(v1, 'toarray') else v1
    dense_matrix2 = v2.toarray().ravel() if hasattr(v2, 'toarray') else v2

    # Convert the sparse matrices to dense arrays
    dense_matrix1 = dense_matrix1.toarray().ravel() if hasattr(dense_matrix1, 'toarray') else dense_matrix1
    dense_matrix2 = dense_matrix2.toarray().ravel() if hasattr(dense_matrix2, 'toarray') else dense_matrix2

    # Compute the cosine similarity between the dense arrays
    similarity = abs(cosine(dense_matrix1, dense_matrix2))

    return similarity

### DB에 있는 질문전부에 대해서, 타겟질문(user질문) 에 대해 score들을 구하고, 최적의 score를 가진 질문을 찾아낸다

### dist가 작을수록 score가 좋은것 == 벡터거리가 가까울수록 유사한 문장

In [21]:
best_q=None
best_dist=65535
best_i=[]
dis_list=[]

for i in range(0,num_samples):
    if i != num_samples-1: #제거한 문장 제외
        post_vec=vec.getrow(i)
        #print(type(post_vec))

        dis=dist_raw_eculidian(post_vec, target_q_vec) #함수변경가능 유클리디언 or 코사인유사도
        #print("==Post %i번질문 with dist=%.3f : %s" % (i,dis,contents[i]))

        if dis < best_dist:
            best_dist=dis
            best_i.append(i)


print('\n\n\n\n----------------------------------------------------------')
print('토크나이징된 타겟질문:',tok_sentence_forVectorize[-1])
print('\n\n\n\n----------------------------------------------------------')
print("Best recommendation question is %i번질문 -> %s, dist = %.3f" % (best_i[-1],contents[best_i[-1]],best_dist))
print("Best recommendation question is %i번질문 -> %s, dist = %.3f" % (best_i[-2],contents[best_i[-2]],best_dist))
print("Best recommendation question is %i번질문 -> %s, dist = %.3f" % (best_i[-3],contents[best_i[-3]],best_dist))





----------------------------------------------------------
토크나이징된 타겟질문:  무선 키 보드 키 스킨 교체 또는 신규 신청 애데 헌 조치 내용 은 어떻게도 ㅣ 나요 ?




----------------------------------------------------------
Best recommendation question is 5번질문 -> 무선키보드 키스킨 교체 또는 신규 신청에 대한 조치내용은 어떻게 되나요?, dist = 1.077
Best recommendation question is 1번질문 -> 무선키보드 교체 또는 신규 신청에 대한 조치내용은 어떻게 되나요?, dist = 1.077
Best recommendation question is 0번질문 -> questions, dist = 1.077


In [22]:
for i in range(0,num_samples):
    if i != num_samples-1: #제거한 문장 제외
        post_vec=vec.getrow(i)
        #print(type(post_vec))

        dis=dist_raw_eculidian(post_vec, target_q_vec) #함수변경가능 유클리디언 or 코사인유사도
        print("==Post %i번질문 with dist=%.3f : %s" % (i,dis,contents[i]))

        if dis < best_dist:
            best_dist=dis
            best_i.append(i)

==Post 0번질문 with dist=1.414 : questions
==Post 1번질문 with dist=1.148 : 무선키보드 교체 또는 신규 신청에 대한 조치내용은 어떻게 되나요?
==Post 2번질문 with dist=1.237 : 유선키보드 교체 또는 신규 신청에 대한 조치내용은 어떻게 되나요?
==Post 3번질문 with dist=1.219 : 무선마우스 교체 또는 신규 신청에 대한 조치내용은 어떻게 되나요?
==Post 4번질문 with dist=1.284 : 유선키보드 마우스 세트 교체 또는 신규 신청에 대한 조치내용은 어떻게 되나요?
==Post 5번질문 with dist=1.077 : 무선키보드 키스킨 교체 또는 신규 신청에 대한 조치내용은 어떻게 되나요?
==Post 6번질문 with dist=1.160 : 유선키보드 키스킨 교체 또는 신규 신청에 대한 조치내용은 어떻게 되나요?
==Post 7번질문 with dist=1.338 : 미니어음교환스캐너 케이블 교체 또는 신규신청에 대한 조치내용은 어떻게 되나요?
==Post 8번질문 with dist=1.259 : 키보드 받침대 교체 또는 신규 신청에 대한 조치내용은 어떻게 되나요?
==Post 9번질문 with dist=1.334 : 스마트스캐너 신분증트레이 교체 또는 신규신청에 대한 조치내용은 어떻게 되나요?
==Post 10번질문 with dist=1.306 : 핀패드 케이블 교체 또는 신규신청에 대한 조치내용은 어떻게 되나요?
==Post 11번질문 with dist=1.148 : 무선키보드 교체 또는 신규 신청에 대한 조치내용은 어떻게 되나요?
==Post 12번질문 with dist=1.237 : 유선키보드 교체 또는 신규 신청에 대한 조치내용은 어떻게 되나요?
==Post 13번질문 with dist=1.219 : 무선마우스 교체 또는 신규 신청에 대한 조치내용은 어떻게 되나요?
==Post 14번질문 with dist=1.284 : 유선키보드 마우스 세트 교체 또는 신규 